In [5]:
# Imports and definitions
import os
import numpy as np
from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D
from keras.layers import Dropout, Flatten, Dense, BatchNormalization, Activation
from keras import optimizers
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint 
from keras.preprocessing.image import ImageDataGenerator
from PIL import ImageFile                            
ImageFile.LOAD_TRUNCATED_IMAGES = True  #To address OSError: image file is truncated 

# Testing Xception implementation.
from keras.applications import inception_resnet_v2

# Paths to images 
training_path = 'dogImages/train'
validation_path = 'dogImages/valid'
test_path = 'dogImages/test'

ImportError: cannot import name 'inception_resnet_v2'

I'm going to use transfer learning for this. The following steps create the bottleneck features for the Xception CNN with data augmentation. 

generate_bottelneck_features_n_labels(generator, batch_size, model): Takes in the data generator and model, and it returns the bottleneck_features and the corresponding labels.

In [2]:
# Since Keras doesn't seem to give an option to get labels with predict_generator, doing it manually. 
def generate_bottelneck_features_n_labels(generator, batch_size, model):
    ind = 0
    list_batches = []
    list_labels = []
    for images, label in generator:
        bott_features = model.predict(images)
        list_batches.append(bott_features)
        list_labels.append(label)
        if generator.samples//batch_size+1 <= ind:
            break
        ind += 1
    bottleneck_features = np.vstack(list_batches)
    labels = np.vstack(list_labels)
    return bottleneck_features, labels

In [3]:
if (not os.path.isfile('bottleneck_features/bf_data_aug_inception_train.npz')) \
    or (not os.path.isfile('bottleneck_features/bf_data_aug_inception_validation.npz')) \
    or (not os.path.isfile('bottleneck_features/bf_data_aug_inception_test.npz')):
    
    
    batch_size = 32
    # Instanciate the inception model. Don't include the FC layers.
    inception_model = inception_resnet_v2.InceptionResNetV2(include_top=False, weights='imagenet')

    # Data augmentation on the training data.
    data_generator_train = ImageDataGenerator(
                                        rescale=1./255,
                                        zoom_range=0.2,
                                        rotation_range=10,
                                        width_shift_range=0.1,
                                        height_shift_range=0.1,
                                        horizontal_flip=True)

    # data generator for validation & test data.
    data_generator_test = ImageDataGenerator(rescale=1./255)

    train_generator = data_generator_train.flow_from_directory(training_path, 
                                                        target_size=(224, 224), 
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        shuffle=False)

    validation_generator = data_generator_test.flow_from_directory(validation_path, 
                                                        target_size=(224, 224), 
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        shuffle=False)

    test_generator = data_generator_test.flow_from_directory(test_path, 
                                                        target_size=(224, 224), 
                                                        batch_size=batch_size,
                                                        class_mode='categorical',
                                                        shuffle=False)
    
    
    train_inception, labels_train_inception = generate_bottelneck_features_n_labels(train_generator, batch_size, inception_model)
    valid_inception, labels_valid_inception = generate_bottelneck_features_n_labels(validation_generator, batch_size, inception_model)
    test_inception, labels_test_inception = generate_bottelneck_features_n_labels(test_generator, batch_size, inception_model)
    
    np.savez(open('bottleneck_features/bf_data_aug_inception_train.npz', 'wb'), train_inception=train_inception, labels_train_inception=labels_train_inception)
    np.savez(open('bottleneck_features/bf_data_aug_inception_validation.npz', 'wb'), valid_inception=valid_inception, labels_valid_inception=labels_valid_inception)
    np.savez(open('bottleneck_features/bf_data_aug_inception_test.npz', 'wb'), test_inception=test_inception, labels_test_inception=labels_test_inception)
    
else:
    train_data_inception = np.load('bottleneck_features/bf_data_aug_inception_train.npz')
    train_inception = train_data_inception['train_inception']
    labels_train_inception = train_data_inception['labels_train_inception']
    
    valid_data_inception = np.load('bottleneck_features/bf_data_aug_inception_validation.npz')
    valid_inception = valid_data_inception['valid_inception']
    labels_valid_inception = valid_data_inception['labels_valid_inception']
    
    test_data_inception = np.load('bottleneck_features/bf_data_aug_inception_test.npz')
    test_inception = test_data_inception['test_inception']
    labels_test_inception = test_data_inception['labels_test_inception']

KeyboardInterrupt: 

Now that I have the bottleneck features, create the NN for the FC layers.

In [ ]:
my_model = Sequential()
my_model.add(GlobalAveragePooling2D(input_shape=train_inception.shape[1:]))
# my_model.add(Dense(1000, activation='relu'))
# my_model.add(BatchNormalization())
# my_model.add(Dropout(0.2))
# my_model.add(Dense(500, activation='relu'))
# my_model.add(BatchNormalization())
my_model.add(Dropout(0.3))
my_model.add(Dense(133, activation='softmax'))
my_model.summary()

Define optimizer, loss function and metric.

In [ ]:
opt = optimizers.Adam(lr=0.002)
my_model.compile(loss='categorical_crossentropy', optimizer=opt, metrics=['accuracy'])

Train the model.

In [ ]:
epochs = 10
batch_size = 32

checkpointer = ModelCheckpoint(filepath='saved_models/weights.best.xception_transfer.hdf5', 
                               verbose=1, save_best_only=True)

my_model.fit(train_inception, labels_train_inception, 
          validation_data=(valid_inception, labels_valid_inception),
          epochs=epochs, batch_size=batch_size, callbacks=[checkpointer], verbose=1)

Load the best weights for the model.

In [ ]:
my_model.load_weights('saved_models/weights.best.xception_transfer.hdf5')

Accuracy over test data set.

In [ ]:
# Expanding dimensions for the input features, adding 1 for the num of samples. 
# So it matches with the expected input from Keras
# Using argmax to retrieve the index of the dog breed with maximum probability.
inception_predictions = [np.argmax(my_model.predict(np.expand_dims(tensor, axis=0))) for tensor in test_inception]

# report test accuracy
# Checking indexes from predictions and test labels.
test_accuracy = 100*np.sum(np.array(inception_predictions)==np.argmax(labels_test_inception, axis=1))/len(inception_predictions)
print('Test accuracy: %.4f%%' % test_accuracy)

In [ ]:
def get_top_breeds(predicted_vector, dog_names, threshold):
    mask = (predicted_vector >= threshold)
    predict_mask = mask*predicted_vector
    indeces = np.argsort(predict_mask, axis=-1)
    relationship = OrderedDict()
    total = 0
    for value in reversed(indeces[0, :]):
        if float(predict_mask[0, value]) == 0.0:
            break
        relationship[dog_names[value]] = predict_mask[0, value]
        total += predict_mask[0, value]
        
    # Rescale prob to sum 100%
    for key in relationship:
        prob = relationship[key]
        relationship[key] = prob/total * 100
    return relationship

In [ ]:
def Xception_predict_breed(img_path, threslhold=0.005):
    # extracting bottleneck features
    tensor = path_to_tensor(img_path)
    bottleneck_features_inference = extract_Xception(tensor)
    # obtain predicted probability vector
    predicted_vector = my_model.predict(bottleneck_features_inference)
    dog_breeds = get_top_breeds(predicted_vector, dog_names, threslhold)
    # return dog breed that is predicted by the model
    return dog_breeds